LLaMA stands for Large Language Model Meta AI ,it’s a family of open-source transformer models developed by Meta (Facebook AI Research). It’s designed to be efficient, open, and highly performant.

In [ ]:
!pip install transformers accelerate sentencepiece #installing dependencies

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM # Casual Language Model (trained left to right in comparison to bert thats bidirectional)
import torch

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch


model_id = "distilgpt2"

# Load tokenizer and model

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id, output_hidden_states=True)
model.eval();

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/762 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/353M [00:00<?, ?B/s]

The following generation flags are not valid and may be ignored: ['output_hidden_states']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

#Code for converting txt files into 3 word chunks

In [ ]:
with open("/content/Copy of walk_in_the_park_396wds_scanner_confirmed.txt", "r") as f:
     story = f.read()

In [ ]:
import re
import pandas as pd

def conversion(text):
    text = text.replace('\n', ' ') # Remove line breaks (data cleaning)
    text = ' '.join(text.split())  #remove multiple spaces
    words = text.split()

    chunks = []
    i = 0
    while i < len(words):
        chunk = words[i:i+3]
        chunk_str = ' '.join(chunk)
        chunks.append(chunk_str)
        i += 3

    return chunks

chunks = conversion(story)

# Preview
for chunk in chunks[:10]:
    print(chunk)

﻿It was fall,
and I embarked
on a stroll
through an area
near my home.
I came across
many beautiful things,
and the cool
breeze was invigorating.
It's nice to


In [ ]:
# Save to CSV
df = pd.DataFrame({'chunk': chunks})
df.to_csv("walk_in_park.csv", index=False)

# Extracting embeddings

In [ ]:
import pandas as pd

df = pd.read_csv("/content/walk_in_park.csv")
word_list = df["chunk"].tolist()
print(word_list[:5])  # Preview

['\ufeffIt was fall,', 'and I embarked', 'on a stroll', 'through an area', 'near my home.']


In [ ]:
word_list[0] = word_list[0].lstrip('\ufeff')

In [ ]:
import torch

def get_gpt2_embedding_with_context(chunk_idx, chunks, context_size=10):
    context_start = max(0, chunk_idx - context_size)
    context_window = chunks[context_start:chunk_idx + 1]
    full_input = ' '.join(context_window)

    # Tokenize
    inputs = tokenizer(full_input, return_tensors="pt", truncation=True)
    inputs = {k: v.to(model.device) for k, v in inputs.items()}

    with torch.no_grad():
        outputs = model(**inputs)

    # Get hidden states from middle layer (layer 6 of 12)
    hidden_states = outputs.hidden_states
    middle_layer = hidden_states[6].squeeze(0)  # [seq_len, hidden_dim]

    target_ids = tokenizer(chunks[chunk_idx], return_tensors='pt')['input_ids'].squeeze()
    target_len = target_ids.shape[0]

    target_embeddings = middle_layer[-target_len:]
    avg_embedding = target_embeddings.mean(dim=0)

    return avg_embedding.cpu().numpy()

In [ ]:
#getting embeddings for each chunk
all_embeddings = []
for i in range(len(chunks)):
    try:
        emb = get_gpt2_embedding_with_context(i, chunks)
        all_embeddings.append(emb)
    except Exception as e:
        print(f"Error at chunk {i}: {e}")
        all_embeddings.append(np.zeros((768,)))

In [ ]:
import numpy as np

np.save('gpt2_walkpark.npy', np.array(all_embeddings))